In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('merged_data_fillna_timeid.csv')
df

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen,ResidualNoWinsorCumReturn winsorized,RawNoWinsorCumReturn winsorized,estVol winsorized,SharesOutstanding winsorized,MDV_63 winsorized,CleanMid winsorized,CumVolume winsorized
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446039,20171229,17:30:00.000,ID00GVXK059,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0,1.0,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0
2446040,20171229,17:30:00.000,ID00HFWT7M4,0.000843,-0.005238,0.104843,170.875,55438068.0,234.749070,18108.0,1.0,0.000843,-0.005238,0.104843,170.875,55438068.0,216.695569,18108.0
2446041,20171229,17:30:00.000,ID00HRW4917,0.033315,0.031948,0.192046,27.126,2810998.5,58.018875,13716.0,1.0,0.024701,0.031163,0.192046,27.126,2810998.5,58.018875,13716.0
2446042,20171229,17:30:00.000,ID00HTQM455,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0,1.0,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0


In [6]:
def compute_features(df):
    # intraday return features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17res'
    features = temp.reset_index()

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # 17:30 ma raw return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3raw'] = features['17ma1raw'] - features['17ma3raw']
    features['17ma1-5raw'] = features['17ma1raw'] - features['17ma5raw']
    features['17ma1-20raw'] = features['17ma1raw'] - features['17ma20raw']

    # 17:30 ma residual return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3res'] = features['17ma1res'] - features['17ma3res']
    features['17ma1-5res'] = features['17ma1res'] - features['17ma5res']
    features['17ma1-20res'] = features['17ma1res'] - features['17ma20res']

    # market value
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id') * df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'market_value'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # intraday volume features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='16:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') - df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')) / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # turnover ma
    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma1'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma3'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma5'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma20'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # original features 
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='estVol winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'estVol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid16'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid10'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'vol17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # Normalization
    features.iloc[:,2:] = (features.iloc[:,2:] - features.iloc[:,2:].mean())/features.iloc[:,2:].std()

    # Merging with target
    target = df[df["Time"] == "17:30:00.000"][['Date','Id','ResidualNoWinsorCumReturn winsorized']].copy()
    merged = features.merge(target ,on =['Date','Id']).drop_duplicates()
    merged = merged.sort_values(['Id','Date'],ascending = [True, True])
    merged["y"] = merged.groupby(['Id'])["ResidualNoWinsorCumReturn winsorized"].shift(-1)
    merged = merged.dropna(how = 'any')
    merged = merged.sort_values('Date',ascending=True)
    merged = merged.drop(["Id", "Date","ResidualNoWinsorCumReturn winsorized"], axis = 1)
    merged.to_csv('beforeTrainning.csv')
    return merged

In [7]:
m = compute_features(df)
m

,10/17res,10/17raw,16/17res,16/17raw,17ma1raw,17ma3raw,17ma5raw,17ma20raw,17ma1-3raw,17ma1-5raw,...,turnover_ma1,turnover_ma3,turnover_ma5,turnover_ma20,estVol,cleanMid17,cleanMid16,cleanMid10,vol17,y
19,-1.577538,0.345493,0.861773,0.266602,-0.170505,-0.473889,1.645554,0.363198,0.129827,-1.008020,...,0.515209,1.155765,2.703089,2.681067,1.350592,-0.112409,-0.111202,-0.108973,-0.346780,-0.017619
521779,0.715504,1.555173,0.255538,-0.418209,-0.829787,-0.463027,-2.047258,-0.788749,-0.683878,0.098985,...,0.244704,0.047523,0.143052,-0.155459,-0.141847,-0.491071,-0.492062,-0.483048,0.750851,-0.000857
523534,0.765668,1.734385,0.549001,0.204154,-1.109391,-0.098172,-0.019000,-1.326247,-1.286062,-1.221166,...,-0.550656,-0.548128,-0.558232,-0.821417,0.099609,-0.657418,-0.657124,-0.652050,-0.475643,0.005427
524559,0.025304,1.134664,0.829231,-0.213477,-0.523002,-0.623993,-0.647421,1.549304,-0.193969,-0.257537,...,-0.372846,-0.597798,-0.680498,-0.594521,0.828204,-0.703546,-0.703791,-0.700701,-0.343368,0.022712
525278,-0.985300,0.559126,-0.741003,-1.021690,0.352473,-0.031412,-1.074016,-0.227556,0.453379,0.926416,...,-0.499692,-0.628027,-0.689018,-0.692489,0.109363,-0.431191,-0.433955,-0.427895,-0.535061,-0.001233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474687,-0.342521,-0.245514,-0.549003,-0.108364,0.017619,0.138282,0.203656,1.265525,-0.077072,-0.081335,...,-0.723052,-0.773898,-0.731602,-0.427851,-0.835283,1.696570,1.694950,1.689080,-0.613384,0.005219
620200,-0.595339,-0.449335,-1.269106,-0.434007,0.023501,0.205612,-0.136688,0.026831,-0.117931,0.094649,...,-0.633173,-0.728051,-0.619775,-0.401951,-1.177794,1.121951,1.116933,1.111062,-0.590321,0.001534
11908,0.284356,0.348269,0.668955,0.983363,0.091175,0.563927,0.534569,0.972526,-0.290911,-0.164466,...,-0.045735,-0.029692,-0.106161,0.126363,-0.382933,0.216092,0.222397,0.221003,-0.590369,0.001472
719403,1.048000,0.480620,0.634301,0.383708,-0.720090,0.535678,1.188106,-0.281275,-1.262499,-1.390156,...,-0.687117,-0.858123,-0.784079,-0.738903,-0.194376,-0.617501,-0.616863,-0.615790,-0.510486,0.001981
